In [ ]:
import geopandas as gpd
from shapely.geometry import mapping
from pystac_client import Client
import os
import urllib.request
import numpy as np
import rasterio
from glob import glob
import matplotlib.pyplot as plt
import rioxarray

In [ ]:
# EODC STAC API and collection
api_url = "https://stac.eodc.eu/api/v1"
collection_id = "GFM"
asset_names = ["ensemble_flood_extent"]
download_root_path = "./gfm_data/"

In [ ]:
# Load your AOI (assumes one polygon)
gdf = gpd.read_file(r"Data\Data\baringo_bbox\baringo_bbox.shp")
aoi_geom = mapping(gdf.geometry[0])

In [ ]:
catalog = Client.open(api_url)
print(f"Searching GFM data for {time_range}...")
search = catalog.search(
    collections=[collection_id],
    intersects=aoi_geom,
    datetime=time_range,
    max_items=100,
)
items = search.item_collection()
print(f"Found {len(items)} items for {time_range}.")

In [ ]:
# Time range: one month
time_range = "2018-01-01/2022-02-01"


# Search STAC API
catalog = Client.open(api_url)
print(f"🔎 Searching GFM data for {time_range}...")
search = catalog.search(
    collections=[collection_id],
    intersects=aoi_geom,
    datetime=time_range,
    max_items=100
)

items = search.item_collection()
print(f"Found {len(items)} items for Jan 2018.")

# Download assets
for item in items:
    item_path = os.path.join(download_root_path, item.collection_id, item.id)
    os.makedirs(item_path, exist_ok=True)
    for asset_name in asset_names:
        asset = item.assets.get(asset_name)
        if asset and "data" in asset.roles:
            out_file = os.path.join(item_path, os.path.basename(asset.href))
            urllib.request.urlretrieve(asset.href, out_file)

print("Test download complete.")

In [ ]:
import os
import numpy as np
import rasterio
from glob import glob

# Find raster paths
tif_paths = glob("./gfm_data/GFM/**/*.tif", recursive=True)
print(f"Found {len(tif_paths)} rasters")

# Initialize
sum_array = None
meta = None

for i, path in enumerate(tif_paths):
    with rasterio.open(path) as src:
        data = src.read(1)
        mask = (data != src.nodata) & (data > 0)  # Only valid flooded pixels
        if sum_array is None:
            sum_array = np.zeros_like(data, dtype=np.uint16)
            meta = src.meta.copy()
        sum_array[mask] += 1  # Increment flood count only where flooded

# Write output
meta.update(dtype='uint16', count=1)
with rasterio.open("./gfm_month/flood_extent.tif", 'w', **meta) as dst:
    dst.write(sum_array, 1)

print("Flood extent saved .")


In [ ]:
import matplotlib.pyplot as plt
import rioxarray
# flood_frequency is your xarray DataArray (uint16 flood counts)
tiff_path = "./flood_extent.tif"

flood_extent = rioxarray.open_rasterio(tiff_path).squeeze()
plt.figure(figsize=(6, 6))
flood_extent.plot(
    cmap="Blues",
    vmin=0,
    vmax=flood_extent.max().item(),
    cbar_kwargs={"label": "Flood extent"},
)
plt.title("Flood extent 2018 January")
plt.show()


In [ ]:
# Current date
today = datetime.utcnow().date()

# Yearly loop
for year in range(2018, 2026):
    if year == today.year:
        end_date = today.strftime("%Y-%m-%d")
    else:
        end_date = f"{year}-12-31"
    time_range = f"{year}-01-01/{end_date}"
    
    #print(f"\n Searching for {year} (up to {end_date})...")

    search = catalog.search(
        collections=[collection_id],
        intersects=aoi_geom,
        datetime=time_range,
        max_items=1000
    )

    items = search.item_collection()
    print(f" Found {len(items)} items for {year}")

    for item in items:
        item_path = os.path.join(download_root_path, str(year), item.collection_id, item.id)
        os.makedirs(item_path, exist_ok=True)
        for asset_name in asset_names:
            asset = item.assets.get(asset_name)
            if asset and "data" in asset.roles:
                out_file = os.path.join(item_path, os.path.basename(asset.href))
                print(f"Downloading {out_file}")
                urllib.request.urlretrieve(asset.href, out_file)

print("Downloads complete.")